# Estimating whether a stock will go up, down, or hold based on news headlines

We will use a Random Forest Classifier, a Naive-Bayes classifier with a "bag of words" approach, and an LSTM Recurrent Neural Network to see which gives us the best performance.

LABELS: 

0 -> Stocks will go down

1 -> Stocks will go up or stay the same

In [1]:
# Import numpy and pandas to start
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read in our first dataset as df
df = pd.read_csv('/content/drive/MyDrive/StockStalker/Stock_Dataa.csv', encoding='ISO-8859-1')

In [4]:
# Look at first 5 rows
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United (England),Women in record South Pole walk,Vasco da Gama (Brazil),South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,Revenge is sweet for jubilant Cronje,"Our choice, not theirs",Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,"Thuggery, Toxins and Ties",Met faces fresh attack on race crime,Everton fans top racist 'league of shame',"Our breasts, ourselves",Russia's new boss has an extremely strange his...,Always and forever,Most everywhere: UDIs,Most wanted: Chloe lunettes,

In [5]:
# Look at last 5 rows
df.tail()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,Weapons shipped into Jordan by the CIA and Sau...,Angela Merkel said the U.K. must file exit pap...,In a birth offering hope to a threatened speci...,Sky News Journalist Left Speechless As Leave M...,Giant panda in Macau gives birth to twins,Get out now: EU leader tells Britain it must i...,Sea turtle 'beaten and left for dead' on beach...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
4097,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,UK's Osborne: 'Absolutely' going to have to cu...,'Do not let Scotland down now' : Scottish MEP ...,British pound could hit history-making dollar ...,"Merkel vows to strengthen EU, tells UK no 'che...","""Ryanair will not deploy new aircraft on route...","People, ever more greedy and stupid, destroy t...",Siemens freezes new UK wind power investment f...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
4098,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,Iceland historian Johannesson wins presidentia...,99-Million-Yr-Old Bird Wings Found Encased in ...,A chatbot programmed by a British teenager has...,The Philippine president-elect said Monday he ...,Former Belgian Prime Minister ridicules Nigel ...,Brexiteer Nigel Farage To EU: 'You're Not Laug...,Islamic State bombings in southern Yemen kill ...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
4099,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says p

Data preprocessing for our text data:

In [6]:
# Remove punctuation
data = df.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Also, I'm going to rename our columns for ease of access
list1 = [i for i in range(25)]
new_index = [str(i) for i in list1]
data.columns = new_index
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,Hammers hand Robson a youthful lesson,Saints party like it s,Wear wolves have turned into lambs,Stump mike catches testy Gough s taunt,Langer escapes to hit,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,A tale of two tails,I say what I like and I like what I say,Elbows Eyes and Nipples,Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite
2,Coventry caught on counter by Flo,United s rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United England,Women in record South Pole walk,Vasco da Gama Brazil,South Melbourne Australia,Necaxa Mexico,Real Madrid Spain,Raja Casablanca Morocco,Corinthians Brazil,Tony s pet project,Al Nassr Saudi Arabia,Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,Auntie back in fashion even if the dress look...,Shoaib appeal goes to the top,Hussain hurt by shambles but lays blame on e...,England s decade of disasters,Revenge is sweet for jubilant Cronje,Our choice not theirs,Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for,Christmas glitches,Upending a table Chopping a line and Scoring ...,Scientific evidence unreliable defence claims,Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake up of failing NHS,Lessons of law s hard heart
4,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers are you all whingers,Hollywood Beyond,Ashes and diamonds,Whingers a formidable minority,Alan Parker part two,Thuggery Toxins and Ties,Met faces fresh attack on race crime,Everton fans top racist league of shame,Our breasts ourselves,Russia s new boss has an extremely strange his...,Always and forever,Most everywhere UDIs,Most wanted Chloe lunettes,Return of the cane completely off the agenda,From Sleepy Hollow to Greeneland,Blunkett outlines vision for over s,Embattled Dobson attacks play now pay later ...,Doom and the Dome,What is the north south divide,Aitken released from jai

In [7]:
# Converting the headlines into lowercase
# so all words are treated the same (apple and Apple, for example)
for index in new_index:
  data[index] = data[index].str.lower()
data.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [8]:
# Combining the first 25 headlines of first record
' '.join(str(x) for x in data.iloc[1,0:25])

'scorecard the best lake scene leader  german sleaze inquiry cheerio  boyo the main recommendations has cubie killed fees  has cubie killed fees  has cubie killed fees  hopkins  furious  at foster s lack of hannibal appetite has cubie killed fees  a tale of two tails i say what i like and i like what i say elbows  eyes and nipples task force to assess risk of asteroid collision how i found myself at last on the critical list the timing of their lives dear doctor irish court halts ira man s extradition to northern ireland burundi peace initiative fades after rebels reject mandela as mediator pe points the way forward to the ecb campaigners keep up pressure on nazi war crimes suspect jane ratcliffe yet more things you wouldn t know without the movies millennium bug fails to bite'

In [9]:
# Combining the headlines for the rest of our records so we can convert them into vectors
headlines = []
for row in range(0, len(data.index)):
  headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))


df['Headlines'] = headlines

data = df[['Label', 'Headlines']]
data

,Label,Headlines
0,0,a hindrance to operations extracts from the...
1,0,scorecard the best lake scene leader german s...
2,0,coventry caught on counter by flo united s riv...
3,1,pilgrim knows how to progress thatcher facing ...
4,1,hitches and horlocks beckham off but united su...
...,...,...
4096,0,barclays and rbs shares suspended from trading...
4097,1,scientists to australia if you want to ...
4098,1,explosion at airport in istanbul yemeni former...
4099,1,jamaica proposes marijuana dispensers for tour...


In [10]:
from sklearn.model_selection import train_test_split

# Set our X and y
# X -> features
# y -> target

X = data['Headlines']
y = data["Label"]

print(len(data))
print(len(y))

# Perform our train test split using sklearn's train_test_split function
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(len(X_train))
print(len(y_train))

4101
4101
3075
3075


## Natural Language Processing

Using 'bag of words' model and a TF-IDF Vectorizer for converting text data into vectors. 

For both Random Forest Classifier & Naive-Bayes Classifier

In [11]:
# Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [12]:
# Implementing Bag of Words model: 
countvector = CountVectorizer(ngram_range=(2, 2))
train_dataset = countvector.fit_transform(X_train) # Converting all the headlines to vectors

In [13]:
# Data converts into sparse matrix 
train_dataset[0]

<1x478938 sparse matrix of type '<class 'numpy.int64'>'
	with 112 stored elements in Compressed Sparse Row format>

First, we'll use our bag of words model with Random Forest Classifier

In [14]:
# Implement random forest classifier
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_dataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
# Predicting for our test dataset
test_dataset = countvector.transform(X_test)
predictions= random_classifier.predict(test_dataset)

In [16]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [17]:
# For calculating our accuracy
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [18]:
matrix= confusion_matrix(y_test,predictions)
print(matrix)
score= accuracy_score(y_test,predictions)
print(score)
report= classification_report(y_test,predictions)
print(report)

[[ 19 490]
 [ 14 503]]
0.5087719298245614
              precision    recall  f1-score   support

           0       0.58      0.04      0.07       509
           1       0.51      0.97      0.67       517

    accuracy                           0.51      1026
   macro avg       0.54      0.51      0.37      1026
weighted avg       0.54      0.51      0.37      1026



It looks like our overall accuracy is around 51% with this approach. 

The notebook I based my initial preprocessing and approach off (cited below, Sun, J.) acheived a nearly 84% accuracy with this approach, although I've split my test and train data somewhat differently. 

In Sun, J.'s approach, the test and train data actually overlapped quite a bit: for the entire year 2015. My concern is with having a model overfit in our production case, and so I opted to use the SciKit Learn's built in train_test_split function. 

In the future, after experimenting with other models and architectures, I will come back and potentially explore other ways of splitting and processing my data, to see if I get better results.

# Using Random Forest Classfier with TF-IDF Vectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [20]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
train_dataset= tfidf.fit_transform(X_train) # Changing all our headlines to vectors using TF-IDF technique

In [21]:
# Implement random forest classfier on train_dataset
random_classifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
random_classifier.fit(train_dataset, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
# Predict on test dataset
test_dataset= tfidf.transform(X_test)
predictions= random_classifier.predict(test_dataset)

In [23]:
predictions

array([1, 1, 1, ..., 0, 1, 1])

In [24]:
# ACCURACY AFTER USING TF-IDF VECTORIZER
matrix= confusion_matrix(y_test,predictions)
print(matrix)
score= accuracy_score(y_test,predictions)
print(score)
report= classification_report(y_test,predictions)
print(report)

[[105 404]
 [ 96 421]]
0.5126705653021443
              precision    recall  f1-score   support

           0       0.52      0.21      0.30       509
           1       0.51      0.81      0.63       517

    accuracy                           0.51      1026
   macro avg       0.52      0.51      0.46      1026
weighted avg       0.52      0.51      0.46      1026



It still looks like our accuracy is quite low. The representation of these models as performing well on this data (see citation below) came from a notebook which split train and test data in such a way it overlapped. I was considering that his model performed well based on overfitting - and not true results, and this hypothesis seems to have been proven. 

I'd like to see if there are ways to optimize the performance of these models for this problem. First, we're going to try our Naive-Bayes Classifier and see how that performs, and then we're going to examine another architecture that may be good for sentiment analysis: an LSTM.

## Naive-Bayes Classifier

First, we'll use our 'bag of words' model to convert text into vectors, before looking at a different vectorization approach.

In [27]:
from sklearn.naive_bayes import MultinomialNB
naive = MultinomialNB()

In [28]:
countvector = CountVectorizer(ngram_range=(2, 2))
train_dataset = countvector.fit_transform(X_train)

In [29]:
# Fitting Naive Bayes Classifier with training data
naive.fit(train_dataset, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
# Predicting on test dataset
test_dataset = countvector.transform(X_test)
predictions = naive.predict(test_dataset)

In [31]:
predictions

array([1, 0, 1, ..., 0, 1, 1])

In [32]:
matrix= confusion_matrix(y_test, predictions)
print(matrix)
score= accuracy_score(y_test, predictions)
print(score)
report= classification_report(y_test, predictions)
print(report)

[[108 401]
 [115 402]]
0.49707602339181284
              precision    recall  f1-score   support

           0       0.48      0.21      0.30       509
           1       0.50      0.78      0.61       517

    accuracy                           0.50      1026
   macro avg       0.49      0.49      0.45      1026
weighted avg       0.49      0.50      0.45      1026



We could try Naive-Bayes with TF-IDF Vectorization, but I'm not convinced that it's necessarily going to perform any better for this problem. I'd like to take a different approach. First, I want to look at LSTM RNN's, and see how one performs on this dataset.

Knowing that LSTM's often need a LOT of data to perform well, I'm then going to see if I can combine the data that I have, preprocess it accordingly, and provide the LSTM with the combined set so it's better able to train. 

First, though, I'm going to just 'prove concept' with the current dataset that we're working with, and see if our accuracy even marginally improves. It's possible another type of architecture altogether suits us better, so let's see!

# LSTM with bag of words model & limited data

In [33]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import numpy as np
import re

In [34]:
# Set parameters
# I'm going to try, first, giving the model our headlines vectorization

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)
X = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(X)

In [ ]:
# Compose the LSTM network.
# embed_dim, lstm_out, batch_size, and dropout_x are hyperparameters that may need to be tweaked in order to optimize output
# input_length, which we pass to our Embedding layer, takes the 1st index of our X value -> or, the length of our features?
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 601, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 601, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 197       
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
print(type(X))
print(type(y_train))
model.fit(X, y_train, epochs = 15, batch_size=batch_size, verbose = 2)

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
Epoch 1/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 2/15
97/97 - 304s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 3/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 4/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 5/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 6/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 7/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 8/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 9/15
97/97 - 305s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 10/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 11/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 12/15
97/97 - 306s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 13/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 14/15
97/97 - 307s - loss: 0.0000e+00 - accuracy: 0.5265
Epoch 15/15
97/97 - 308s - loss: 0.0000e+00 - accuracy: 0.5265


# LSTM with TF-IDF Vectorization and increased data

In [ ]:
# I'm going to start from scratch and change the way I preprocess my data to better suit the LSTM model's learning.

# I'm going to keep reddit to maybe use for testing.
reddit = pd.read_csv("/content/drive/MyDrive/StockStalker/RedditNews.csv").drop('Date', axis=1)
print(reddit.head())

# Both of these datasets are labeled and contain top 25 headlines, so I'm going to combine them
combined = pd.read_csv("/content/drive/MyDrive/StockStalker/Combined_News_DJIA.csv").drop('Date', axis=1)
print(combined.head())
stock_data = pd.read_csv("/content/drive/MyDrive/StockStalker/Stock_Dataa.csv", encoding='ISO-8859-1').drop('Date', axis=1)
print(stock_data.head())

print(f"Shape combined: {combined.shape}")
print(f"Shape stock_data: {stock_data.shape}")

# Since they have the same number of columns, we can concatenate them so we have one larger dataframe to work with
# TODO: do we need to separate labels 0 and 1 for LSTM? 

                                                News
0  A 117-year-old woman in Mexico City finally re...
1   IMF chief backs Athens as permanent Olympic host
2  The president of France says if Brexit won, so...
3  British Man Who Must Give Police 24 Hours' Not...
4  100+ Nobel laureates urge Greenpeace to stop o...
   Label  ...                                              Top25
0      0  ...           b"No Help for Mexico's Kidnapping Surge"
1      1  ...  b"So this is what it's come to: trading sex fo...
2      0  ...  b"BBC NEWS | Asia-Pacific | Extinction 'by man...
3      0  ...  b'2006: Nobel laureate Aleksander Solzhenitsyn...
4      1  ...  b'Philippines : Peace Advocate say Muslims nee...

[5 rows x 26 columns]
   Label  ...                         Top25
0      0  ...            Recovering a title
1      0  ...  Millennium bug fails to bite
2      0  ...                  Useful links
3      1  ...   Lessons of law's hard heart
4      1  ...                    Gone aloft

[5 r

In [ ]:
# Because two of our datasets have the same # of columns and the same columns, we're going to stack them.
data = pd.concat([combined, stock_data], axis=0, ignore_index=True)

In [ ]:
# Combine top 25 headlines of each record for vectorizer
headlines = []
for row in range(0, len(data.index)):
  headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

In [ ]:
# Take a look at the result:
print(headlines[0])

0 b"Georgia 'downs two Russian warplanes' as countries move to brink of war" b'BREAKING: Musharraf to be impeached.' b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)' b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire' b"Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing" b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side" b"The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]' b'Did the U.S. Prep Georgia for War with Russia?' b'Rice Gives Gree

# LSTM with combined datasets

Citations:

Sun, J. (2016, August). Daily News for Stock Market Prediction, Version 1. Retrieved (2021, March) from https://www.kaggle.com/aaron7sun/stocknews.